In [28]:
import numpy as np
from numpy import pi
from sympy import symbols, integrate, solve, latex, simplify, exp, I
from IPython.display import display, Math
from pandas import DataFrame
from plotly.express import line


h = 4.136 * 10**(-15) # in eV*s
h_bar = h/(2*pi) # in eV/Hz
m_e = 9.109 * 10**(-31) # in kg

# 1.
Assume that $\varphi_n \left(x \right)$ are the eigenfunctions in an infinite square well $( \rightarrow\infty \text{ for } x<0 \text{ and } x>a)$. Calculate the overlap integrals
$\int_{0}^{a}{{\varphi_n}^\ast\left(x\right)\varphi_m\left(x\right)dx.}$

(Hint: The eigenfunctions to different eigenvalues are supposed to be orthogonal). (434: 6 pts, 591: 4 pts)


In quantum mechanics, eigenfunctions (AKA eigenstates) represent states corresponding to physically measurable values, (where the measured values are the eigenvalues).  That is, they each correspond to a single solution to the Schrodinger equation, the superposition of which gives the wavefunction.  But since we never observe an individual particle to be in a superposition of states, only ever in a single state at once (although measuring many particles with the same wavefunction reproduces the superposition), it's necesarry that eigenstates with different eigenvalues be mutually exclusive, which requires they be mutually orthogonal.  Hence, there can be no nonzero overlap between eigenstates with different eigenvalues.  For eigenstates with the same eigenvalue, the overlap will of course be 1 or 100%.

# 2.
For an infinite potential well of width $15$ nm, compute the energies associated with the transitions between the five lowest levels. Give the answers in eV. (434: 10 pts, 591: 5 pts)

For a particle trapped in an infinite square well, the energy level is quantized due to the boundary conditions applies to the Schrodinger equation.  The energy levels are given by the equation:
$E_n = \frac{\hbar^2 n^2 \pi^2}{2mw^2}$

In [3]:
def infinite_well_energy_transition(n2, n1):
  w = 15 * 10**(-9)
  def infinite_well_energy(n):
    return (h_bar**2 * n**2 * pi**2)/(2 * m_e * w**2) #using regular electron mass,
                                                      #since problem doesn't specify particle type
  return infinite_well_energy(n2) - infinite_well_energy(n1)

transitions = ['5-4', '4-3', '3-2', '2-1']
for i in range(0,4):
  temp = infinite_well_energy_transition(5-i, 4-i)*1.602*10**(-19)
  #times 1.6022*10^(-19) to convert hbar to be in terms of Joules then the answer back to eV
  display(Math(f'\Delta E_n({transitions[i]}) = '
               f'{round(temp,3)}\\text{{ eV}} = '
               f'{round(10**(3) * temp, 3)} \\text{{ meV}}'))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# 3.
For a finite potential well of width 20 nm and height 0.2 eV (use an effective mass appropriate to GaAs electrons, m = 0.067m0), compute the energy needed to ionize an electron (move it from the lowest energy level to the top of the well). If a 0.3 eV photon excites the electron out of the well, what is its kinetic energy? What is its wavelength? (434: 12 pts, 591: 8 pts)

Inside a finite well, a particle has the wavefunction

$\psi(x)_1 = D\cos(lx) \tag{1}$

where

$l = \frac{\sqrt{2m(V_0 - E)}}{\hbar} \tag{2}$.

Outside the well,

<p>$\psi(x)_2 = F\exp(-kx) \tag{3}$</p>

where

$k = \frac{\sqrt{2mE}}{\hbar} \tag{4}$.


Since the wavefunction and its derivative must be continuous, we set $\psi(w)_1 = \psi(w)_2$ and $\psi'(w)_1 = \psi'(w)_2$, which gives:

<p>$D\cos(lw) =  F\exp(-kw)\tag{5}$</p>

and

<p>$-lD\sin(lw) =  -kF\exp(-kw)\tag{6}$</p>

Deviding (5) by (6) gives:

<p>$k = l\tan(lw)\tag{7}$</p>

Since $k$ and $l$ are both functions of $E$, we can find the allowed energy values from this equation.  However, it's not convenient to have two separate functions of $E$.  Instead, following the derivation used in Griffiths' *Quantum Mechanics* in section 2.6, set:


<p>$z = lw \tag{8}$</p> 
    
    
<p>$((7) \text{ and } (8) \implies \tan z = \frac{k}{l} \tag{9})$</p>

and

$z_0 = \frac{w}{\hbar}\sqrt{2mV_0}\tag{10}$.

Additionally, from the definitions of $k$ and $l$ it follows that

$k^2+l^2 = \frac{2mV_0}{\hbar^2} \implies k = \sqrt{\frac{2mV_0}{\hbar^2} - l^2} = \sqrt{\left(\frac{z_0}{w}\right)^2 - \left(\frac{z}{w}\right)^2} \implies kw = \sqrt{z_0^2 - z^2}\tag{11}$.

$(11) \text{ and } (9) \implies \tan z =\sqrt{\left(\frac{z_0}{z}\right)^2-1} \tag{12}$

Now, solving for $E$ in (2) and multiplying by $w$ to get it in terms of $z$ instead of $l$, we get that

<p>$ E = V_0 - \frac{z^2\hbar^2}{2mw^2} \tag{13}$</p>

Now I'll find the solutions for $z$ by plotting the roots of equation 12 and then compute the energy difference between the highest and lowest energy levels.

In [61]:
m = m_e*0.067
V_0 = 0.2 *1.602*10**(-19) #in Joules
w = 20 * 10**(-9) #in meters
z_0 = (w/((h_bar*10**(-9)))) * (2*m*V_0)**0.5

z = np.linspace(-z_0, z_0, 1000)
outputs = np.tan(z) - ((z_0/z)**2 - 1)**0.5
df = DataFrame({'z':z, 'outputs':outputs})

line(df, x = 'z', y = 'outputs')

From the plot, there are 3 roots of equation 12, at around $z=-2$, $z=1.29$ and $z=3.79$.  That means there are three bound states of the electron and the ionization energy will simply be the difference between the lowest and highest energy levels.

In [60]:
def finite_well_energy(z):
  return (V_0 - (z**2 * h_bar**2)/(2 * m * (w * 10**(-9))**2))*1.602*10**(-19)

finite_well_energy(3.79) - finite_well_energy(-2)

-0.014735332434638163

In [62]:
z_0

1.9000425985963274e-09